<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>







<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2022-10-07 00:45:12--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  23.6MB/s    in 9.7s    

2022-10-07 00:45:23 (25.8 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [1]:
#!unzip concrete_data_week4.zip

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [ ]:
# Import Libraries
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define Global Constants

num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [ ]:
# Construct ImageDataGenerator Instances
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
# Training set
 
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [ ]:
# Validation set

validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 9501 images belonging to 2 classes.


Next, we will add the VGG16 pre-trained model to out model. However, note that we don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument include_top and set it to **False**

In [ ]:
# Build, Compile and Fit Model

# Sequential
model = Sequential()

# Vgg16
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

# Dense Activation Layer
model.add(Dense(num_classes, activation='softmax'))

58900480/58889256 [==============================] - 0s 0us/step


Since the VGG16 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense output layer. To do that, we run the following.

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically, that is the number of images divided by the batch size. Therefore, we define our steps per epoch as follows:

In [ ]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [ ]:
# Fit the model with ``fit_generator``

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/2
301/301 [==============================] - 224s 688ms/step - loss: 0.0684 - accuracy: 0.9812 - val_loss: 0.0213 - val_accuracy: 0.9966
Epoch 2/2
301/301 [==============================] - 220s 730ms/step - loss: 0.0160 - accuracy: 0.9966 - val_loss: 0.0132 - val_accuracy: 0.9974


In [ ]:
model.save('classifier_VGG16.h5')

In [ ]:
from keras.applications.resnet import ResNet50, preprocess_input
# Build, Compile and Fit Model

# Sequential
model = Sequential()

# Vgg16
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

# Dense Activation Layer
model.add(Dense(num_classes, activation='softmax'))

94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
model.layers[0].trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Steps and Epochs
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

# Fit the model with ``fit_generator``

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/2
301/301 [==============================] - 140s 450ms/step - loss: 0.0252 - accuracy: 0.9930 - val_loss: 0.0068 - val_accuracy: 0.9984
Epoch 2/2
301/301 [==============================] - 134s 446ms/step - loss: 0.0056 - accuracy: 0.9989 - val_loss: 0.0052 - val_accuracy: 0.9985


In [ ]:
model.save('classifier_ResNet50.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
resnet_model = keras.models.load_model("classifier_ResNet50.h5")
vgg16_model = keras.models.load_model("classifier_VGG16.h5")

In [ ]:
# Testing set
 
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 500 images belonging to 2 classes.


In [ ]:
resnet_loss , resnet_accuracy = resnet_model.evaluate(test_generator)
print("Resnet Loss: ", resnet_loss)
print("Resnet Accuracy: ", resnet_accuracy)

5/5 [==============================] - 3s 339ms/step - loss: 0.0038 - accuracy: 1.0000
Resnet Loss:  0.00384633825160563
Resnet Accuracy:  1.0


In [ ]:
vgg16_loss , vgg16_accuracy = vgg16_model.evaluate(test_generator)
print("VGG16 Loss: ", vgg16_loss)
print("VGG16 Accuracy: ", vgg16_accuracy)

5/5 [==============================] - 3s 554ms/step - loss: 0.0105 - accuracy: 0.9980
VGG16 Loss:  0.010485395789146423
VGG16 Accuracy:  0.9980000257492065


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

In [ ]:
#print(resnet_predictions[:6])
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [ ]:
resnet_predictions = resnet_model.predict(test_generator)

In [ ]:
#print(resnet_predictions[:6])

predicted_class_indices_resnet=np.argmax(resnet_predictions,axis=1)
predictions_resnet = [labels[k] for k in predicted_class_indices_resnet]
print(predictions_resnet[:6])

['positive', 'positive', 'negative', 'negative', 'negative', 'positive']


In [ ]:
vgg16_predictions = vgg16_model.predict(test_generator)

In [ ]:
#print(vgg_predictions[:6])
predicted_class_indices=np.argmax(vgg16_predictions,axis=1)
predictions = [labels[k] for k in predicted_class_indices]
print(predictions[:6])

['positive', 'negative', 'positive', 'negative', 'positive', 'positive']


<a id="item44"></a>

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).